#Ejemplo: Método Desacoplado Rápido para la Solución de Flujos de Potencia


unif-ejemplo1.svg

*Tabla de datos de nodos:*

| Nodos | Tipo de nodo | Tensión nominal (kV) | Potencia activa inyectada (MW) | Potencia reactiva inyectada (MVAr) | Magnitud de voltaje (p.u.) | Ángulo de voltaje (p.u.) |
|-------|--------------|----------------------|--------------------------------|------------------------------------|----------------------------|--------------------------|
| 1     | Referencia   | 100                  | --                             | --                                 | 1.02                       | 0.0                      |
| 2     | Generación   | 100                  | 50                             | --                                 | 1.02                       | 0.0                      |
| 3     | Carga        | 100                  | -100                           | -60                                | 1.02                       | 0.0                      |

*Inicialmente, se toman los datos del sistema de potencia: matriz de admitancias, voltajes y potencias iniciales.*

In [1]:
import numpy as np

def rad(deg):
  # Convertir grados en radianes
  return deg*np.pi/180

def asvoltage(Mag,Ang):
  # Convertir vectores de magnitud y ángulo en vector de fasores
  # Mag: vector de magnitudes de voltaje
  # Ang: vector de ángulos de voltaje
  v=np.zeros((len(Mag),1),dtype=complex)
  for i in range(len(Mag)):
    v[i]=Mag[i]*(np.cos(Ang[i])+1j*np.sin(Ang[i]))
  return v

# Definir matriz de admitancias
n=3
Ybus=np.array([(15-35j,-10+20j,-5+15j),(-10+20j,30-60j,-20+40j),(-5+15j,-20+40j,25-55j)],dtype=complex)
B=np.imag(Ybus)
G=np.real(Ybus)

# Valores definidos de potencia
Pdef=np.array([[0.5,-1]])
Qdef=np.array([[-0.6]])

# Valores iniciales de voltaje
vbusMag=np.array([1.02,1.02,1.02])
vbusAng=np.array([rad(0),rad(0),rad(0)])
vbus=asvoltage(vbusMag,vbusAng)
print('Voltajes:\n',vbus)

Voltajes:
 [[1.02+0.j]
 [1.02+0.j]
 [1.02+0.j]]


Se calculan las potencias inyectadas y los residuos con respecto a las potencias definidas.

In [2]:
# Calcular potencias inyectadas a cada bus
S=np.zeros((n,1),dtype=complex)
for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))

print('Potencia compleja:\n',S)

# Calcular residuos
def residuos(S,Pdef,Qdef):
  Qcal=np.imag(S)
  Pcal=np.real(S)
  dP=np.swapaxes(Pdef,0,1)-np.real(S)[1:3]
  dQ=Qdef-np.imag(S)[2]
  return dP, dQ
[dP,dQ]=residuos(S,Pdef,Qdef)
print('Residuos:',dP,dQ)

Potencia compleja:
 [[1.81188398e-15+3.62376795e-15j]
 [3.62376795e-15+7.24753590e-15j]
 [0.00000000e+00+7.24753590e-15j]]
Residuos: [[ 0.5]
 [-1. ]] [[-0.6]]


Como los residuos son mayores a la tolerancia (supongamos que es $10^{-6}$), se debe continuar a calcular el Jacobiano y actualizar las variables.

In [3]:
# Se definen los nodos con P conocida y Q conocida
pconocida=[2,3]
qconocida=[3]
Qcal=np.imag(S)
Pcal=np.real(S)

# Se calcula el "Jacobiano" desacoplado
B1=np.zeros((len(pconocida),len(pconocida)))
for i in range(len(pconocida)):
  for j in range(len(pconocida)):
    k=pconocida[i]-1
    m=pconocida[j]-1
    B1[i][j]=-np.imag(Ybus[k][m])

B2=np.zeros((len(qconocida),len(qconocida)))
for i in range(len(qconocida)):
  for j in range(len(qconocida)):
    k=qconocida[i]-1
    m=qconocida[j]-1
    B2[i][j]=-np.imag(Ybus[k][m])

dPQ=np.concatenate((dP,dQ),axis=0)

# Ciclo desacoplado
count=0
print('Iter.   Residuo máximo       Voltajes (p.u.)          Ángulos (rad)')
while ((max(abs(dPQ))>10e-6) & (count<20)):
  # Calcular variaciones
  dD=np.linalg.solve(B1, dP)
  dV=np.linalg.solve(B2,dQ)

  # Actualizar voltajes
  for i in range(n-1):
    vbusAng[pconocida[i]-1]+=dD[i]
  for i in range(len(qconocida)):
    vbusMag[qconocida[i]-1]=vbusMag[qconocida[i]-1]*(1+dV[i])
  vbus=asvoltage(vbusMag,vbusAng)

  #Actualizar potencias
  for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))
  Qcal=np.imag(S)
  Pcal=np.real(S)

  #Calular residuos
  [dP,dQ]=residuos(S,Pdef,Qdef)
  dPQ=np.concatenate((dP,dQ),axis=0)

  count+=1
  print(count,'   ',max(dPQ),'    ',vbusMag,' ',vbusAng)

Iter.   Residuo máximo       Voltajes (p.u.)          Ángulos (rad)
1     [0.30552071]      [1.02       1.02       1.00887273]   [ 0.         -0.00735294 -0.02352941]
2     [0.18998012]      [1.02       1.02       1.00068811]   [ 0.         -0.00801947 -0.01845924]
3     [0.09288436]      [1.02       1.02       1.00343065]   [ 0.         -0.00855662 -0.01539571]
4     [0.03233709]      [1.02       1.02       1.00512525]   [ 0.         -0.00826757 -0.01644162]
5     [0.01536544]      [1.02      1.02      1.0045217]   [ 0.         -0.00815222 -0.01707704]
6     [0.00810093]      [1.02      1.02      1.0041731]   [ 0.         -0.00821333 -0.01684211]
7     [0.00392085]      [1.02      1.02      1.0043082]   [ 0.         -0.00823625 -0.01671149]
8     [0.00135024]      [1.02       1.02       1.00437979]   [ 0.         -0.00822275 -0.01676376]
9     [0.00075613]      [1.02       1.02       1.00434979]   [ 0.         -0.0082181  -0.01679055]
10     [0.00036391]      [1.02       1.02       1.

In [5]:
# Potencia inyectada final
print('Potencia activa inyectada \n',Pcal)
print('Potencia reactiva inyectda \n',Qcal)

Potencia activa inyectada 
 [[ 0.50977158]
 [ 0.4999932 ]
 [-0.99999194]]
Potencia reactiva inyectda 
 [[ 0.07095666]
 [ 0.55126304]
 [-0.60000676]]
